<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/notebooks/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install tokenizer
!pip install ekphrasis
!pip install wordninja
!pip install emoji
!pip install spacy_udpipe
!pip install language_tool_python

In [11]:
import pandas as pd
import csv
import nltk
import re
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tokenizer import *
from nltk.corpus import stopwords

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import wordninja

from nltk.stem import SnowballStemmer

import ast

import emoji
import unicodedata

import gzip

import spacy_udpipe
import language_tool_python

import sys
import os

In [16]:
# directory name:
input_dir = '/content/drive/My Drive/HLT/dataset_training/'
output_dir = '/content/drive/My Drive/HLT/clean_dataset_training/'

# Spec
pd.set_option("display.max_colwidth", None)

In [13]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
#tsv_tweets_file = open("/content/drive/My Drive/HLT/dataset/haspeede2_reference_taskAB-tweets.tsv")
#tsv_news_file = open("/content/drive/My Drive/HLT/dataset/haspeede2_reference_taskAB-news.tsv")
tsv_file = open(input_dir+"haspeede2_dev_taskAB.tsv")

#dataset_tweets_raw = pd.read_csv(tsv_tweets_file,sep='\t')
#dataset_tweets_raw.rename(columns={"text ": "text"}, inplace=True)
#dataset_news_raw = pd.read_csv(tsv_news_file,sep='\t')
#dataset_news_raw.rename(columns={"text ": "text"}, inplace=True)
dataset_raw = pd.read_csv(tsv_file,sep='\t')
dataset_raw.rename(columns={"text ": "text"}, inplace=True)

# Preprocessing phase
Following the "Preprocessing" section of https://books.openedition.org/aaccademia/4832?lang=it#tocfrom1n4 we will perform:
*   List item
*   List item



### Extraction of the first feature: length of the comment

In [26]:
def comment_length(text):
    return len(text)

In [27]:
dataset_raw['text_length'] = dataset_raw['text'].apply(comment_length)

Extraction of the second feature: percentage of words written in CAPS-LOCK inside the comment.

In [28]:
def caps_lock_words(text):
    words = text.split()
    count_caps_lock = 0
    number_of_words = len(words)
    
    for word in words:
        if word.isupper() == True:
            count_caps_lock = count_caps_lock + 1
            
    return ((count_caps_lock*100)//number_of_words)

In [32]:
dataset_raw['#C-L words'] = dataset_raw['text'].apply(caps_lock_words)

In [31]:
dataset_raw

,id,text,hs,stereotype,text_length,%CAPS-LOCK words
0,2066,"È terrorismo anche questo, per mettere in uno stato di soggezione le persone e renderle innocue, mentre qualcuno... URL",0,0,120,10
1,2045,@user @user infatti finché ci hanno guadagnato con i campi #rom tutto era ok con #Alemanno #Ipocriti,0,0,101,0
2,61,"Corriere: Tangenti, Mafia Capitale dimenticataMazzette su buche e campi rom URL #roma",0,0,86,8
3,1259,"@user ad uno ad uno, perché quando i migranti israeliti arrivarono in terra di Canaan fecero fuori tutti i Canaaniti.",0,0,118,0
4,949,Il divertimento del giorno? Trovare i patrioti italiani che inneggiano contro i rom facendo la spesa alla #Lidl (multinazionale tedesca).,0,0,138,0
...,...,...,...,...,...,...
6832,9340,"Gli stati nazionali devono essere pronti a rinunciare alla propria sovranità. Lo ha detto la Merkel , che ha aggiunto che gli stati nazionali non devono ascoltare la volontà dei loro cittadini quando si tratta di questioni che riguardano immigrazione, confini, o persino sovranità URL",0,0,285,2
6833,9121,"Il ministro dell'interno della Germania #HorstSeehofer,sta facendo la proposta di dare soldi agli immigrati che vogliono tornare a casa e aiutarli a creare un'attività a casa loro e fare business con la Germania.Chi paga?Una parte i crucchi e il resto l'Europa, cioè io e voi!",0,0,277,0
6834,8549,"#Salvini: In Italia troppi si sono montati la testa, io ringrazio Dio e voi per questi mesi straordinari. Vi raccontavano che su immigrazione non si poteva fare nulla, è bastato usare buonsenso e coraggio. #iocisono #piazzadelpopolo",0,0,233,0
6835,9240,@user @user Chi giubila in buona fede non ha capito niente. Purtroppo credo che i più non siano in buona fede. I migranti sono un grosso business e chi finora li ha voluti non vuole perdere questo guadagno,0,0,206,2
